In [2]:
import cobra
from cobra.io import read_sbml_model

In [3]:
k12 = read_sbml_model("../GEMs/iML1515.xml")
bl21 = read_sbml_model("../GEMs/iHK1487.xml")

In [22]:
bl21.genes.ECD_03417

Gene identifier,ECD_03417
Name,b3565
Memory address,0x12666fe80
Functional,True
In 2 reaction(s),"XYLI2, XYLI1"


In [21]:
for gene in bl21.genes:
    if gene.name == "b3565":
        print(gene.id)

ECD_03417


In [33]:
from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model, write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
from cobra.manipulation import knock_out_model_genes
from GEM_eval import energy_generating_cycle_test
from config import ROOT_DIR


print("... k12 model")
# read model for E. coli K12
k12 = read_sbml_model('../GEMs/iML1515.xml')

print("... universal model")
#read the universal model
uni = read_sbml_model(str(ROOT_DIR / "community_modelling" / "GEMs" / "bigg_universe.xml"))

CAL11 = k12.copy()

############### Knock-outs ###############

# pheA | gene b2599
knock_out_model_genes(CAL11, ["b2599"])
# ptsH | b2415
knock_out_model_genes(CAL11, ["b2415"])
# ptsI | b2416
knock_out_model_genes(CAL11, ["b2416"])
# crr | b2417
knock_out_model_genes(CAL11, ["b2417"])
# ydiB | b1692
knock_out_model_genes(CAL11, ["b1692"])

############### Metabolites ###############

print("adding metabolites")

#p-Coumaric acid
p_coumaric_acid_c = uni.metabolites.get_by_id("T4hcinnm_c")
p_coumaric_acid_c.compartment = "c"

#caffeic acid
caffeic_acid_c = uni.metabolites.get_by_id("34dhcinm_c")
caffeic_acid_c.compartment = "c"

#external caffeic acid
caffeic_acid_e = uni.metabolites.get_by_id("34dhcinm_e")
caffeic_acid_e.compartment = "e"

#ammonia
nh4_c = k12.metabolites.get_by_id("nh4_c")

#tyrosine 
tyr__L_c = k12.metabolites.get_by_id("tyr__L_c")

#NADH
nadh_c = k12.metabolites.get_by_id("nadh_c")

#NAD+
nad_c = k12.metabolites.get_by_id("nad_c")

#H+
h_c = k12.metabolites.get_by_id("h_c")

#ATP
atp_c = k12.metabolites.get_by_id("atp_c")

#AMP
adp_c = k12.metabolites.get_by_id("adp_c")

#Diphosphate
pi_c = k12.metabolites.get_by_id("pi_c")

############### Reactions ###############

print("adding reactions")

#tyr -> p-coumaric acid
TAL = Reaction('TAL')
TAL.name = 'L-tyrosine ammonia-lyase'
TAL.subsystem = 'CA module'
TAL.lower_bound = -1000  # This is the default
TAL.upper_bound = 1000  # This is the default
#L-Tyrosine <=> 4-Coumarate + Ammonia
TAL.add_metabolites({
    tyr__L_c: -1.0,
    p_coumaric_acid_c: 1.0,
    nh4_c: 1.0
})
#TAL.gene_reaction_rule = 'tal' #synthetic rgTAL gene

#p-coumaric acid -> caffeic acid
COURCA = Reaction('COURCA')
COURCA.name = '4-hydroxyphenylacetate 3-hydroxylase'
COURCA.subsystem = 'CA module'
COURCA.lower_bound = -1000  # This is the default
COURCA.upper_bound = 1000  # This is the default
#4-Coumarate <=> Caffeate
COURCA.add_metabolites({
    p_coumaric_acid_c: -1.0,
    nadh_c: -1.0,
    caffeic_acid_c: 1.0,
    nad_c: 1.0,
    h_c: 1.0
})
#COURCA.gene_reaction_rule = '(hpaB and hpaC)' #synthetic hpaC gene

#caffeic acid transport reaction
CAFFAt = Reaction('34DHCINMt')
CAFFAt.name = 'caffeic acid active transport'
CAFFAt.subsystem = 'CA module'
CAFFAt.lower_bound = -1000  # This is the default
CAFFAt.upper_bound = 1000  # This is the default
CAFFAt.add_metabolites({
    caffeic_acid_c: -1.0,
    atp_c: -1.0,
    caffeic_acid_e: 1.0,
    adp_c: 1.0,
    pi_c: 1.0
})

# add reactions to model
CAL11.add_reactions([TAL, COURCA, CAFFAt])

#exhange reactions for CA
#CAL11.metabolites.get_by_id("34dhcinm_e").compartment = 'e' #set compartment to be 'e' rather than 'C_e'
CAL11.add_boundary(CAL11.metabolites.get_by_id("34dhcinm_e"), type="exchange")

#stopping caffeic acid uptake so that the model does not use this as substrate
CAL11.reactions.EX_34dhcinm_e.lower_bound = 0

... k12 model
... universal model


No objective coefficients in model. Unclear what should be optimized


adding metabolites
adding reactions


In [88]:
CAL11.reactions.EX_glc__D_e.lower_bound = 0

In [89]:
CAL11.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.003847,0,0.00%
cl_e,EX_cl_e,0.003847,0,0.00%
cobalt2_e,EX_cobalt2_e,1.848E-05,0,0.00%
cu2_e,EX_cu2_e,0.000524,0,0.00%
fe2_e,EX_fe2_e,0.01187,0,0.00%
k_e,EX_k_e,0.1443,0,0.00%
mg2_e,EX_mg2_e,0.006411,0,0.00%
mn2_e,EX_mn2_e,0.0005107,0,0.00%
mobd_e,EX_mobd_e,5.173E-06,0,0.00%
nh4_e,EX_nh4_e,7.844,0,0.00%


In [34]:
CAL11.reactions.EX_xyl__D_e.lower_bound = -10

In [38]:
CAL11.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.2951,6,100.00%
o2_e,EX_o2_e,1.77,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-1.77,1,100.00%
h2o_e,EX_h2o_e,-1.77,0,0.00%


In [40]:
k12.reactions.EX_glc__D_e.lower_bound = 0
k12.reactions.EX_xyl__D_e.lower_bound = -10

In [43]:
uptake = k12.summary().uptake_flux

In [47]:
uptake_rx = list(uptake.index)

In [52]:
CAL11.reactions.EX_phe__L_e.lower_bound = -10

In [69]:
CAL11.metabolites.glc__D_c.summary()

Percent,Flux,Reaction,Definition
100.00%,10,GLCt2pp,glc__D_p + h_p --> glc__D_c + h_c
Percent,Flux,Reaction,Definition
100.00%,-10,HEX1,atp_c + glc__D_c --> adp_c + g6p_c + h_c


In [59]:
CAL11.reactions.G6PDH2r

Reaction identifier,G6PDH2r
Name,Glucose 6-phosphate dehydrogenase
Memory address,0x1468c4640
Stoichiometry,"g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c D-Glucose 6-phosphate + Nicotinamide adenine dinucleotide phosphate <=> 6-phospho-D-glucono-1,5-lactone + H+ + Nicotinamide adenine dinucleotide phosphate - reduced"
GPR,b1852
Lower bound,-1000.0
Upper bound,1000.0


In [66]:
CAL11.genes.b2415.reactions

frozenset({<Reaction 2DGLCptspp at 0x1479b1570>,
           <Reaction ACGAptspp at 0x1468b48b0>,
           <Reaction ACMANAptspp at 0x1468b4460>,
           <Reaction ACMUMptspp at 0x13bff0430>,
           <Reaction ARBTptspp at 0x1479b2020>,
           <Reaction ASCBptspp at 0x1468b7670>,
           <Reaction CELBpts at 0x144cd41f0>,
           <Reaction CHTBSptspp at 0x1479b86d0>,
           <Reaction DHAPT at 0x1468d86d0>,
           <Reaction FRUpts2pp at 0x14795d150>,
           <Reaction FRUptspp at 0x1468a8ee0>,
           <Reaction GALTptspp at 0x13c014820>,
           <Reaction GAMptspp at 0x14795fca0>,
           <Reaction GLCptspp at 0x13c0502e0>,
           <Reaction MALTptspp at 0x144cfb640>,
           <Reaction MANGLYCptspp at 0x146884f40>,
           <Reaction MANptspp at 0x144cd6a40>,
           <Reaction MNLptspp at 0x146886470>,
           <Reaction SBTptspp at 0x1468aa050>,
           <Reaction SUCptspp at 0x1468bca60>,
           <Reaction TREptspp at 0x1468b9e40>

In [90]:
CAL11.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x1468ea6e0
Stoichiometry,glc__D_e --> D-Glucose -->
GPR,
Lower bound,0
Upper bound,1000.0


In [80]:
from cobra import flux_analysis

sol = CAL11.optimize()

In [ ]:
# ptsH | b2415
knock_out_model_genes(CAL11, ["b2415"])
# ptsI | b2416
knock_out_model_genes(CAL11, ["b2416"])
# crr | b2417
knock_out_model_genes(CAL11, ["b2417"])

In [75]:
from escher import Builder

builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model=CAL11,
)

CYTDK2           0.000000
XPPT             0.000000
HXPRT            0.000000
NDPK5            0.000000
SHK3Dr           0.325055
                   ...   
OCTNLL           0.000000
TAL              0.000000
COURCA           0.000000
34DHCINMt        0.000000
EX_34dhcinm_e    0.000000
Name: fluxes, Length: 2716, dtype: float64

In [87]:
builder.reaction_data = sol.fluxes

In [76]:
builder

Builder()

In [91]:
SAL11 = read_sbml_model("../GEMs/SAL11.xml")
CAL11 = read_sbml_model("../GEMs/CAL11.xml")
MAM3 = read_sbml_model("../GEMs/MAM3.xml")

In [101]:
MAM3.reactions.EX_glc__D_e.lower_bound = -10
MAM3.reactions.EX_xyl__D_e.lower_bound = -10
MAM3.reactions.EX_phe__L_e.lower_bound = -10
MAM3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.2951,6,100.00%
o2_e,EX_o2_e,1.77,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-1.77,1,100.00%
h2o_e,EX_h2o_e,-1.77,0,0.00%


In [99]:
from cobra.manipulation import knock_out_model_genes
knock_out_model_genes(MAM3, ["b3281"])

[]

In [100]:
MAM3.genes.b3281

Gene identifier,b3281
Name,aroE
Memory address,0x12c7a3880
Functional,False
In 1 reaction(s),SHK3Dr


In [103]:
from cobra.io import read_sbml_model
from cobra.manipulation import knock_out_model_genes
import KNOCK_OUTS

SAL11 = read_sbml_model("../GEMs/SAL11.xml")
CAL11 = read_sbml_model("../GEMs/CAL11.xml")
MAM3 = read_sbml_model("../GEMs/MAM3.xml")

knock_out_model_genes(SAL11, KNOCK_OUTS.SAL11_KO)
knock_out_model_genes(MAM3, KNOCK_OUTS.MAM3_KO)
knock_out_model_genes(CAL11, KNOCK_OUTS.CAL11_KO)

[<Reaction FRUptspp at 0x1383ea020>,
 <Reaction GLCptspp at 0x13781d870>,
 <Reaction MANptspp at 0x137f529b0>,
 <Reaction MNLptspp at 0x1291129b0>,
 <Reaction ARBTptspp at 0x128dfea10>,
 <Reaction 2DGLCptspp at 0x128dfd2a0>,
 <Reaction ACGAptspp at 0x1385baad0>,
 <Reaction QUINDHyi at 0x137d9f2e0>,
 <Reaction QUINDH at 0x137e0db10>,
 <Reaction SBTptspp at 0x1383eb370>,
 <Reaction DHAPT at 0x12878fb80>,
 <Reaction SUCptspp at 0x1326153c0>,
 <Reaction ASCBptspp at 0x138363cd0>,
 <Reaction TREptspp at 0x13856ddb0>,
 <Reaction CELBpts at 0x137f53610>,
 <Reaction ACMUMptspp at 0x13781f640>,
 <Reaction PPNDH at 0x132859660>,
 <Reaction MALTptspp at 0x13829beb0>,
 <Reaction GALTptspp at 0x1377b36d0>,
 <Reaction CHTBSptspp at 0x1381b2ef0>,
 <Reaction MANGLYCptspp at 0x12871b760>,
 <Reaction FRUpts2pp at 0x136b12f80>,
 <Reaction GAMptspp at 0x136958fa0>,
 <Reaction ACMANAptspp at 0x1385bb7f0>]

In [112]:
MAM3 = read_sbml_model("../GEMs/MAM3.xml")

In [ ]:
MAM3_KO = ["b2599", "b2415", "b2416", "b2417", "b1692", "b3281"]

In [117]:
MAM3.genes.b3281

Gene identifier,b3281
Name,aroE
Memory address,0x1341501f0
Functional,True
In 1 reaction(s),SHK3Dr


In [118]:
MAM3.metabolites.get_by_id("34dhcinm_e")

Metabolite identifier,34dhcinm_e
Name,"3,4-Dihydroxy-trans-cinnamate"
Memory address,0x1340f0130
Formula,C9H7O4
Compartment,e
In 2 reaction(s),"EX_34dhcinm_e, CAFFAt"


In [111]:
MAM3.reactions.EX_phe__L_e.lower_bound = -10
MAM3.reactions.EX_glc__D_e.lower_bound = -10
MAM3.reactions.EX_xyl__D_e.lower_bound = -10
MAM3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.2951,6,100.00%
o2_e,EX_o2_e,1.77,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-1.77,1,100.00%
h2o_e,EX_h2o_e,-1.77,0,0.00%


In [123]:
print("loading GEMs...")

print("... k12 model")
# read model for E. coli K12
k12 = read_sbml_model(str(ROOT_DIR / "community_modelling" / "GEMs" / "iML1515.xml"))

print("... universal model")
#read the universal model
uni = read_sbml_model(str(ROOT_DIR / "community_modelling" / "GEMs" / "bigg_universe.xml"))

MAM3 = k12.copy()

############### Knock-outs ###############

# # pheA | gene b2599
# knock_out_model_genes(MAM3, ["b2599"])
# # ptsH | b2415
# knock_out_model_genes(MAM3, ["b2415"])
# # ptsI | b2416
# knock_out_model_genes(MAM3, ["b2416"])
# # crr | b2417
# knock_out_model_genes(MAM3, ["b2417"])
# # ydiB | b1692
# knock_out_model_genes(MAM3, ["b1692"])
# # aroE | b3281
# knock_out_model_genes(MAM3, ["b3281"])

#NOTE: might need to knock out more to fully disable glucose uptake / just set glucose transport to 0 or something

############### Metabolites ###############

print("adding metabolites")

#caffeic acid
caffeic_acid_c = uni.metabolites.get_by_id("34dhcinm_c")
caffeic_acid_c.compartment = 'c'

#external caffeic acid
caffeic_acid_e = uni.metabolites.get_by_id("34dhcinm_e")
caffeic_acid_e.compartment = 'e'

#caffeoyl CoA
caffcoa_c = uni.metabolites.get_by_id("caffcoa_c")
caffcoa_c.compartment = 'c'

#3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_c = Metabolite(
    'saa_c',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='c')

#external 3,4-Dihydroxyphenyllactic acid | Salvianic acid A | SAA
saa_e = Metabolite(
    'saa_e',
    formula='C9H10O5',
    name='3,4-Dihydroxyphenyllactic acid',
    compartment='e')

#rosmarinic acid
rosma_c = Metabolite(
    'rosma_c',
    formula='C18H16O8',
    name='Rosmarinic acid',
    compartment='c')

#external rosmarinic acid
rosma_e = Metabolite(
    'rosma_e',
    formula='C18H16O8',
    name='Rosmarinic acid',
    compartment='e')

#ATP
atp_c = k12.metabolites.get_by_id("atp_c")

#AMP
amp_c = k12.metabolites.get_by_id("amp_c")

#Diphosphate
ppi_c = k12.metabolites.get_by_id("ppi_c")

#CoA
coa_c = k12.metabolites.get_by_id("coa_c")

############### Reactions ###############

print("adding reactions")

#caffeic acid transport reaction
CAFFAt = Reaction('CAFFAt')
CAFFAt.name = 'caffa transport'
CAFFAt.subsystem = 'RA module'
CAFFAt.lower_bound = -1000  # This is the default
CAFFAt.upper_bound = 1000  # This is the default
CAFFAt.add_metabolites({
    caffeic_acid_c: -1.0,
    caffeic_acid_e: 1.0
})

CAFFCOA = uni.reactions.get_by_id("CAFFCOA")


#SAA transport reaction
SAAt = Reaction('SAAt')
SAAt.name = 'caffa transport'
SAAt.subsystem = 'RA module'
SAAt.lower_bound = -1000  # This is the default
SAAt.upper_bound = 1000  # This is the default
SAAt.add_metabolites({
    saa_c: -1.0,
    saa_e: 1.0
})


#salvianic acid A + caffeoyl CoA -> Rosmarinic acid
RAS = Reaction('RAS')
RAS.name = 'rosmarinic acid synthase'
RAS.subsystem = 'RA module'
RAS.lower_bound = -1000  # This is the default
RAS.upper_bound = 1000  # This is the default
#Caffeoyl-CoA + 3-(3,4-Dihydroxyphenyl)lactate <=> CoA + Rosmarinate
RAS.add_metabolites({
    caffcoa_c: -1.0,
    saa_c: -1.0,
    coa_c: 1.0,
    rosma_c: 1.0
})
#RAS.gene_reaction_rule = '(ras)' #synthetic 4CL gene


#RA transport reaction
RAt = Reaction('RAt')
RAt.name = 'caffa transport'
RAt.subsystem = 'RA module'
RAt.lower_bound = -1000  # This is the default
RAt.upper_bound = 1000  # This is the default
RAt.add_metabolites({
    rosma_c: -1.0,
    rosma_e: 1.0
})

# add reactions to model
MAM3.add_reactions([CAFFAt, CAFFCOA, SAAt, RAS, RAt])

#exhange reactions for SA, caffa, and RA

#MAM3.metabolites.get_by_id("34dhcinm_e").compartment = 'e' #set compartment to be 'e' rather than 'C_e'

MAM3.add_boundary(MAM3.metabolites.get_by_id("34dhcinm_e"), type="exchange")
MAM3.add_boundary(MAM3.metabolites.get_by_id("saa_e"), type="exchange")
MAM3.add_boundary(MAM3.metabolites.get_by_id("rosma_e"), type="exchange")

#so that cell cannot eat rosmarinic acid
MAM3.reactions.EX_rosma_e.lower_bound = 0

loading GEMs...
... k12 model


... universal model


No objective coefficients in model. Unclear what should be optimized


adding metabolites
adding reactions


In [135]:
# # pheA | gene b2599
knock_out_model_genes(MAM3, ["b2599"])
MAM3.reactions.EX_phe__L_e.lower_bound = -10

MAM3.reactions.EX_xyl__D_e.lower_bound = -10
# # ptsH | b2415
knock_out_model_genes(MAM3, ["b2415"])
# # ptsI | b2416
knock_out_model_genes(MAM3, ["b2416"])
# # crr | b2417
knock_out_model_genes(MAM3, ["b2417"])
# # ydiB | b1692
knock_out_model_genes(MAM3, ["b1692"])
# # 
knock_out_model_genes(MAM3, ["b3281"])

[<Reaction SHK3Dr at 0x16242c580>]

In [137]:
MAM3.genes.b1692

Gene identifier,b1692
Name,ydiB
Memory address,0x1634fb730
Functional,False
In 3 reaction(s),"SHK3Dr, QUINDH, QUINDHyi"


In [144]:
MAM3.reactions.EX_tyr__L_e = -10
MAM3.reactions.EX_thr__L_e = -10

In [145]:
MAM3.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.2951,6,100.00%
o2_e,EX_o2_e,1.77,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-1.77,1,100.00%
h2o_e,EX_h2o_e,-1.77,0,0.00%


In [138]:
MAM3.genes.b3281

Gene identifier,b3281
Name,aroE
Memory address,0x1634fb850
Functional,False
In 1 reaction(s),SHK3Dr


In [136]:
    # ydiB | b1692
    knock_out_model_genes(MAM3, ["b1692"])
    # aroE | b3281
    knock_out_model_genes(MAM3, ["b3281"])

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.2951,6,100.00%
o2_e,EX_o2_e,1.77,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-1.77,1,100.00%
h2o_e,EX_h2o_e,-1.77,0,0.00%
